In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import stats
import seaborn as sns; sns.set()
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

in_p= pd.read_csv('../data/Medicare_Provider_Charge_Inpatient_DRGALL_FY2015.csv')

In [19]:
in_p.columns = ['DRG', 'ID', 'Name', 'Adress', 'City', 'State', 'Zipcode', 'Region', 'Discharges', 'Average_Covered_Charges', 'Average_Total_Payments', 'Average_Medicare_Payments']

In [20]:
in_p.head()

,DRG,ID,Name,Adress,City,State,Zipcode,Region,Discharges,Average_Covered_Charges,Average_Total_Payments,Average_Medicare_Payments
0,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,10033,UNIVERSITY OF ALABAMA HOSPITAL,619 SOUTH 19TH STREET,BIRMINGHAM,AL,35233,AL - Birmingham,11,1014783.45,171081.18,141193.91
1,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,30103,MAYO CLINIC HOSPITAL,5777 EAST MAYO BOULEVARD,PHOENIX,AZ,85054,AZ - Phoenix,28,382051.00,194081.39,167511.96
2,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50025,UNIVERSITY OF CALIFORNIA SAN DIEGO MEDICAL CENTER,200 WEST ARBOR DRIVE,SAN DIEGO,CA,92103,CA - San Diego,21,971881.48,324050.71,294419.62
3,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50100,SHARP MEMORIAL HOSPITAL,7901 FROST ST,SAN DIEGO,CA,92123,CA - San Diego,18,1517857.61,257177.33,233532.83
4,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50108,"SUTTER MEDICAL CENTER, SACRAMENTO",2825 CAPITOL AVENUE,SACRAMENTO,CA,95816,CA - Sacramento,17,1470341.06,346150.12,338472.00


In [23]:
in_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201876 entries, 0 to 201875
Data columns (total 12 columns):
DRG                          201876 non-null object
ID                           201876 non-null int64
Name                         201876 non-null object
Adress                       201876 non-null object
City                         201876 non-null object
State                        201876 non-null object
Zipcode                      201876 non-null int64
Region                       201876 non-null object
Discharges                   201876 non-null int64
Average_Covered_Charges      201876 non-null float64
Average_Total_Payments       201876 non-null float64
Average_Medicare_Payments    201876 non-null float64
dtypes: float64(3), int64(3), object(6)
memory usage: 18.5+ MB


In [1]:
count(in_p.Name.unique())

NameError: name 'count' is not defined

#### _Isolating TN providers and grouping by zipcode and DRG to create a sample case for the dashboard_

In [30]:
tn = in_p.copy()

In [33]:
tn = tn[tn['State'].str.contains("TN")]

In [44]:
tn.head()

,DRG,ID,Name,Adress,City,State,Zipcode,Region,Discharges,Average_Covered_Charges,Average_Total_Payments,Average_Medicare_Payments
58,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,440039,VANDERBILT UNIVERSITY HOSPITAL,1161 21ST AVENUE SOUTH,NASHVILLE,TN,37232,TN - Nashville,19,691394.16,188368.58,186899.63
59,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,440048,BAPTIST MEMORIAL HOSPITAL,6019 WALNUT GROVE ROAD,MEMPHIS,TN,38120,TN - Memphis,15,852935.27,180182.00,179350.67
60,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,440082,SAINT THOMAS WEST HOSPITAL,"4220 HARDING RD, PO BOX 380",NASHVILLE,TN,37205,TN - Nashville,11,976766.73,198706.73,198363.09
418,003 - ECMO OR TRACH W MV >96 HRS OR PDX EXC FA...,440002,JACKSON-MADISON COUNTY GENERAL HOSPITAL,620 SKYLINE DRIVE,JACKSON,TN,38301,TN - Jackson,32,371592.88,117792.13,113317.38
419,003 - ECMO OR TRACH W MV >96 HRS OR PDX EXC FA...,440006,TRISTAR SKYLINE MEDICAL CENTER,3441 DICKERSON PIKE,NASHVILLE,TN,37207,TN - Nashville,21,418145.95,86571.33,74437.29


In [34]:
tn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5122 entries, 58 to 201865
Data columns (total 12 columns):
DRG                          5122 non-null object
ID                           5122 non-null int64
Name                         5122 non-null object
Adress                       5122 non-null object
City                         5122 non-null object
State                        5122 non-null object
Zipcode                      5122 non-null int64
Region                       5122 non-null object
Discharges                   5122 non-null int64
Average_Covered_Charges      5122 non-null float64
Average_Total_Payments       5122 non-null float64
Average_Medicare_Payments    5122 non-null float64
dtypes: float64(3), int64(3), object(6)
memory usage: 520.2+ KB


In [41]:
tn.DRG.value_counts()

871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC                      85
194 - SIMPLE PNEUMONIA & PLEURISY W CC                                        81
690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC                               79
193 - SIMPLE PNEUMONIA & PLEURISY W MCC                                       77
190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC                             76
291 - HEART FAILURE & SHOCK W MCC                                             76
292 - HEART FAILURE & SHOCK W CC                                              72
392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC                  71
603 - CELLULITIS W/O MCC                                                      70
191 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC                              70
641 - MISC DISORDERS OF NUTRITION,METABOLISM,FLUIDS/ELECTROLYTES W/O MCC      69
872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W/O MCC                    69
683 - RENAL FAILURE W CC    

In [43]:
tn.to_csv('tn.csv', encoding='utf-8')